# More Advanced Functional Models

Using the site <https://www.tensorflow.org/guide/keras/functional?hl=en> as a guide to practice coding up more complex models.

## Using a single graph of layers to make multiple models

Using the same stack of layers to instantiate two models: an `encoder` and an end-to-end `autoencoder` model.

In [8]:
import tensorflow as tf

print(tf.__version__)

2.3.0


In [9]:
# Creating th model layers

encoder_input = tf.keras.Input(shape=(28,28,1), name="img_input")

# Creating various layers
x = tf.keras.layers.Conv2D(16,3, activation="relu")(encoder_input)
x = tf.keras.layers.Conv2D(32,3, activation="relu")(x)
x = tf.keras.layers.MaxPool2D(3)(x)
x = tf.keras.layers.Conv2D(32,3, activation="relu")(x)
x = tf.keras.layers.Conv2D(16,3, activation="relu")(x)
encoder_output =  tf.keras.layers.GlobalMaxPool2D()(x)

# Create the first model for encoder
encoder = tf.keras.Model(encoder_input, encoder_output, name="encoder")

# Using the encoder model output for the next layer
x = tf.keras.layers.Reshape((4,4,1))(encoder_output)
x = tf.keras.layers.Conv2DTranspose(16,3,activation="relu")(x) 
x = tf.keras.layers.Conv2DTranspose(32,3,activation="relu")(x) 
x = tf.keras.layers.UpSampling2D(3)(x) 
x = tf.keras.layers.Conv2DTranspose(16,3,activation="relu")(x)
decoder_output = tf.keras.layers.Conv2DTranspose(1,3,activation="relu")(x)

# Define the second model
auto_encoder = tf.keras.Model(encoder_input, decoder_output,name="auto_encoder")

In [10]:
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_input (InputLayer)       [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d_2 (Glob (None, 16)                0   

In [11]:
auto_encoder.summary()

Model: "auto_encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_input (InputLayer)       [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d_2 (Glob (None, 16)               

### Notes for above

The reverse of a `Conv2D` layers is `Conv2DTranspose` layer, and the reverse of a `MaxPooling2D` layer is an `UpSampling2D` layers

## Chaining Different Models together

Using the same structure as above, but creating an `encoder`, a `decoder`, and chain them to obtain the `autoencoder` model.

In [13]:
# Creating the same model layers as above

encoder_input = tf.keras.Input(shape=(28,28,1), name="img_input")

# Creating various layers
x = tf.keras.layers.Conv2D(16,3, activation="relu")(encoder_input)
x = tf.keras.layers.Conv2D(32,3, activation="relu")(x)
x = tf.keras.layers.MaxPool2D(3)(x)
x = tf.keras.layers.Conv2D(32,3, activation="relu")(x)
x = tf.keras.layers.Conv2D(16,3, activation="relu")(x)
encoder_output =  tf.keras.layers.GlobalMaxPool2D()(x)

# Create the first model for encoder
encoder = tf.keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

# Using the encoder model output for the next layer
# This is the change
decoder_input = tf.keras.Input(shape=(16,), name="encoded_img")
x = tf.keras.layers.Reshape((4,4,1))(decoder_input) # Change here as well
x = tf.keras.layers.Conv2DTranspose(16,3,activation="relu")(x) 
x = tf.keras.layers.Conv2DTranspose(32,3,activation="relu")(x) 
x = tf.keras.layers.UpSampling2D(3)(x) 
x = tf.keras.layers.Conv2DTranspose(16,3,activation="relu")(x)
decoder_output = tf.keras.layers.Conv2DTranspose(1,3,activation="relu")(x)

# Define a seperate decoder model
decoder = tf.keras.Model(decoder_input, decoder_output,name="decoder")
decoder.summary()

auto_encoder_input = tf.keras.Input(shape=(28,28,1), name="img_autoencoder")
encoded_img = encoder(auto_encoder_input)
decoded_img = decoder(encoded_img)
auto_encoder = tf.keras.Model(auto_encoder_input, decoded_img, name="autoencoder")
auto_encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_input (InputLayer)       [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d_3 (Glob (None, 16)                0   

### Notes from above

The main point for the summary is to note that the `autoencoder` contains the sub-models for the `encoder` and `decoder`.  The summary does not show all of the sub-models but rather references them by name.  

#### IMP!  The shapes

You will notice that the shapes for the `encoder` input is (none, 28,28,1) and the `encoder` output is (none, 16).  This lines up with the inputs for each of the models in the summary



### Why do all of this?

This leads into *ensembled* models.

In [15]:
def get_model():
    inputs = tf.keras.Input(shape=(128,))
    outputs = tf.keras.layers.Dense(1)(inputs)
    return tf.keras.Model(inputs, outputs)

model1 = get_model()
model2 = get_model()
model3 = get_model()

inputs = tf.keras.Input(shape=(128,))
y1 = model1(inputs)
y2 = model2(inputs)
y3 = model3(inputs)
outputs = tf.keras.layers.average([y1,y2,y3])
ensemble_model = tf.keras.Model(inputs=inputs, outputs=outputs)
ensemble_model.summary()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
functional_7 (Functional)       (None, 1)            129         input_8[0][0]                    
__________________________________________________________________________________________________
functional_9 (Functional)       (None, 1)            129         input_8[0][0]                    
__________________________________________________________________________________________________
functional_11 (Functional)      (None, 1)            129         input_8[0][0]                    
______________________________________________________________________________________